In [1]:
import torch
import numpy as np

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [14]:
x = torch.from_numpy(inputs)
y = torch.from_numpy(targets)
print(x)
print(y.shape)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
torch.Size([5, 2])


In [4]:
w = torch.randn(2, 3, requires_grad = True)
b = torch.randn(2, requires_grad = True)
print(w)
print(b)

tensor([[ 1.0272, -0.3089, -2.3446],
        [ 0.9729, -1.1320, -2.3774]], requires_grad=True)
tensor([1.0127, 0.0589], requires_grad=True)


In [51]:
def model(x, w, b):
    return x @ w.t() + b

In [52]:
p = model(x, w, b)
print(p)

tensor([[ 66.3393,  80.6528],
        [ 71.5724,  91.6398],
        [128.1335, 136.8674],
        [ 71.7192,  95.4560],
        [ 54.0700,  69.3373]], grad_fn=<AddBackward0>)


In [7]:
def mse(p, y):
    diff = p - y
    return torch.sum(diff*diff)/diff.numel()

In [8]:
loss = mse(p, y)

In [9]:
loss.backward()

In [11]:
print(w.grad)
print(b.grad)

tensor([[-11481.0488, -14220.0176,  -8626.8789],
        [-19299.2910, -22992.4531, -13949.9023]])
tensor([-142.4727, -236.0614])


In [13]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [65]:
def train(x, y, lr = 1e-5, epochs=1000):

    w = torch.randn(2,3, requires_grad = True)
    b = torch.randn(2, requires_grad = True)

    for i in range(epochs):
        p = model(x, w, b)
        loss = mse(p, y)
        loss.backward()
        if (i%100 == 0):
            print("loss=", loss)

        with torch.no_grad():
            w -= w.grad * lr      
            b -= b.grad * lr
            w.grad.zero_()
            b.grad.zero_()

In [66]:
train(x,y)

loss= tensor(44698.7734, grad_fn=<DivBackward0>)
loss= tensor(102.1238, grad_fn=<DivBackward0>)
loss= tensor(31.5002, grad_fn=<DivBackward0>)
loss= tensor(11.2624, grad_fn=<DivBackward0>)
loss= tensor(5.2349, grad_fn=<DivBackward0>)
loss= tensor(3.2543, grad_fn=<DivBackward0>)
loss= tensor(2.4586, grad_fn=<DivBackward0>)
loss= tensor(2.0362, grad_fn=<DivBackward0>)
loss= tensor(1.7525, grad_fn=<DivBackward0>)
loss= tensor(1.5359, grad_fn=<DivBackward0>)


# work in batches

In [90]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [71]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [74]:
tensor_ds = TensorDataset(inputs, targets)
tensor_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [77]:
batch_size = 5
train_ds = DataLoader(tensor_ds, batch_size, shuffle=True)
train_ds

In [79]:
for xb, yb in train_ds:
    print(xb, yb)
    break

tensor([[ 91.,  88.,  64.],
        [ 73.,  66.,  44.],
        [103.,  43.,  36.],
        [ 92.,  87.,  64.],
        [ 91.,  87.,  65.]]) tensor([[ 81., 101.],
        [ 57.,  69.],
        [ 20.,  38.],
        [ 82., 100.],
        [ 80., 102.]])


In [89]:
mod = nn.Linear(3, 2)
print(mod.weight)
print(mod.bias)
list(model.parameters())

Parameter containing:
tensor([[ 0.5746,  0.2619,  0.1298],
        [-0.0172,  0.4999, -0.0127]], requires_grad=True)
Parameter containing:
tensor([-0.3728, -0.0612], requires_grad=True)


[Parameter containing:
 tensor([[-0.3905,  0.3873, -0.1928],
         [-0.4073, -0.4559, -0.4565]], requires_grad=True),
 Parameter containing:
 tensor([0.3040, 0.3563], requires_grad=True)]

In [98]:
loss_func = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

steps:
* define model, loss function, optimizer and hyperparams
* for batches:
 * prediction
 * compute loss
 * compute gradients
 * update parameters
 * set gradients to zero

In [103]:
def train(tensor_ds, num_epochs, model, loss_func, opt):
    
    for epoch in range(num_epochs):
        
        for xb, yb in tensor_ds:
            pb = model(xb)
            loss = loss_func(pb, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        if (epoch%10 == 0):
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [104]:
train(tensor_ds, 50, model, loss_func, opt)

Epoch [1/50], Loss: 21.4581
Epoch [11/50], Loss: 14.7747
Epoch [21/50], Loss: 10.2252
Epoch [31/50], Loss: 7.0858
Epoch [41/50], Loss: 4.9150


In [106]:
pred = model(inputs)
print(pred)

tensor([[ 57.0932,  70.8586],
        [ 81.1462,  99.9358],
        [119.8267, 135.0544],
        [ 21.3683,  38.7052],
        [100.0698, 116.8512],
        [ 55.8300,  69.7626],
        [ 80.8476,  99.8576],
        [120.0337, 135.5619],
        [ 22.6315,  39.8011],
        [101.0344, 117.8690],
        [ 56.7946,  70.7804],
        [ 79.8831,  98.8399],
        [120.1253, 135.1325],
        [ 20.4037,  37.6874],
        [101.3330, 117.9471]], grad_fn=<AddmmBackward>)
